In [1]:
import requests 
import pandas as pd
import json
import pickle
import nltk
import string

In [84]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
import pandas as pd
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.naive_bayes import BernoulliNB
from keras.models import load_model
import numpy as np
import re
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
from nltk.stem import WordNetLemmatizer

In [85]:
train = pd.read_csv('train.csv')
train.dropna(inplace = True)
X = train['TEXT']
y = train['truth']

vectorizer = TfidfVectorizer()
vectors_train = vectorizer.fit_transform(X)

tokenizer = Tokenizer(lower=True, char_level=False)
tokenizer.fit_on_texts(train["TEXT"].tolist())

rfc = pickle.load(open('RandomForest.sav', 'rb'))
gbc = pickle.load(open('Gradient-Boost.sav', 'rb'))
etc = pickle.load(open('Extra-Trees.sav', 'rb'))
bnb = pickle.load(open('BernoulliNB.sav', 'rb'))
cnn = load_model('CNN.h5')

In [86]:
def remove_punctuation(text):
    no_punct = "".join([c for c in text if c not in string.punctuation])
    return no_punct

tokenicer = RegexpTokenizer(r'\w+')

def remove_stopwords(text):
    words = [w for w in text if w not in stopwords.words('english')]
    return words

lemmatizer = WordNetLemmatizer()

def word_lemmatizer(text):
    lem_text = " ".join([lemmatizer.lemmatize(i) for i in text])
    return lem_text

In [87]:
teamToken = "zJNrIv24ExPOrOmGw65tG3Ht6rCY4wfbifE6hcVU87U"
get = 'https://erisk.irlab.org/challenge-t1/getwritings/'+teamToken
post= 'https://erisk.irlab.org/challenge-t1/submit/'+teamToken+'/'

In [ ]:
main = []

nick = "nick"
dec = "decision"
sc = "score"
count = 1

r = requests.get(url = get)
data = r.json()
for d in data:
    main.append([d['content'], d['nick']])
maindf = pd.DataFrame(main, columns = ['content', 'nick'])

maindf['content'] = maindf['content'].apply(lambda x: remove_punctuation(x), 1)
maindf['content'] = maindf['content'].apply(lambda x: tokenicer.tokenize(x.lower()))
maindf['content'] = maindf['content'].apply(lambda x: remove_stopwords(x))
maindf['content'] = maindf['content'].apply(lambda x: word_lemmatizer(x))
maindf['content'] = maindf['content'].apply(lambda elem: re.sub(r"(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)|^rt|http.+?", "", elem))  
maindf['content'] = maindf['content'].apply(lambda elem: re.sub(r"\d+", "", elem))


In [97]:

print('Got Sequence',count, "\n")

print("\nTotal Number of Users: 423")
    
while(len(data) != 0):
    vectors = vectorizer.transform(maindf['content'])
    tempcnn = tokenizer.texts_to_sequences(maindf['content'])    
    tempcnn = pad_sequences(tempcnn, maxlen=200)    
    for i in range(5):
        send = []
        if(i == 0):
            print('\nProcessing Run Bernoulli')
            decision = bnb.predict(vectors)
            prob = bnb.predict_proba(vectors)
            score = [max(p) for p in prob]
        elif(i == 1):    
            print('Processing Run Random Forest')
            decision = rfc.predict(vectors)
            prob = rfc.predict_proba(vectors)
            score = [max(p) for p in prob]
        elif(i == 2):  
            print('Processing Run Gradient Boosting')
            decision = gbc.predict(vectors)
            prob = gbc.predict_proba(vectors)
            score = [max(p) for p in prob]
        elif(i == 3):    
            print('Processing Run CNN')
            predictions = cnn.predict(tempcnn, batch_size=1024, verbose=1)
            y_pred = np.argmax(predictions, axis=1)
            decision = [1 if x == 0 else 0 for x in y_pred]
            score = [max(p) for p in predictions]
        elif(i == 4):    
            print('Processing Run Extra Trees')
            decision = etc.predict(vectors)
            prob = etc.predict_proba(vectors)
            score = [max(p) for p in prob]

        for k in range(len(maindf)):
                send.append({nick:str(maindf['nick'][k]),
                              dec:int(decision[k]),
                              sc:float(score[k])})
        
        requests.post(url = post+str(i), json = send)
        
    r = requests.get(url = get)
    
    count += 1
    print('\nGot Sequence ', count)
    
    data = r.json()
    
    templ = []
    tempdf = pd.DataFrame()

    for d in data:
           templ.append([d['content'], d['nick']])
    tempdf = pd.DataFrame(templ, columns = ['content', 'nick'])
    
    tempdf['content'] = tempdf['content'].apply(lambda x: remove_punctuation(x), 1)
    tempdf['content'] = tempdf['content'].apply(lambda x: tokenicer.tokenize(x.lower()))
    tempdf['content'] = tempdf['content'].apply(lambda x: remove_stopwords(x))
    tempdf['content'] = tempdf['content'].apply(lambda x: word_lemmatizer(x))
    tempdf['content'] = tempdf['content'].apply(lambda elem: re.sub(r"(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)|^rt|http.+?", "", elem))  
    tempdf['content'] = tempdf['content'].apply(lambda elem: re.sub(r"\d+", "", elem))
    
    print("\nNumber of users data retrieved:", len(data))
       
    for i1, r1 in maindf.iterrows():
        for i2, r2 in tempdf.iterrows():
            if(r1['nick'] == r2['nick']):
                r1['content'] = r1['content'] + r2['content']

Got Sequence 1364 


Total Number of Users: 423

Processing Run Bernoulli
Processing Run Random Forest
Processing Run Gradient Boosting
Processing Run CNN
423/423 [==============================] - 0s 161us/step
Processing Run Extra Trees

Got Sequence  1365

Number of users data retrieved: 6

Processing Run Bernoulli
Processing Run Random Forest
Processing Run Gradient Boosting
Processing Run CNN
423/423 [==============================] - 0s 202us/step
Processing Run Extra Trees

Got Sequence  1366

Number of users data retrieved: 6

Processing Run Bernoulli
Processing Run Random Forest
Processing Run Gradient Boosting
Processing Run CNN
423/423 [==============================] - 0s 219us/step
Processing Run Extra Trees

Got Sequence  1367

Number of users data retrieved: 6

Processing Run Bernoulli
Processing Run Random Forest
Processing Run Gradient Boosting
Processing Run CNN
423/423 [==============================] - 0s 190us/step
Processing Run Extra Trees

Got Sequence  1368

Nu

Processing Run Gradient Boosting
Processing Run CNN
423/423 [==============================] - 0s 175us/step
Processing Run Extra Trees

Got Sequence  1398

Number of users data retrieved: 6

Processing Run Bernoulli
Processing Run Random Forest
Processing Run Gradient Boosting
Processing Run CNN
423/423 [==============================] - 0s 213us/step
Processing Run Extra Trees

Got Sequence  1399

Number of users data retrieved: 6

Processing Run Bernoulli
Processing Run Random Forest
Processing Run Gradient Boosting
Processing Run CNN
423/423 [==============================] - 0s 186us/step
Processing Run Extra Trees

Got Sequence  1400

Number of users data retrieved: 6

Processing Run Bernoulli
Processing Run Random Forest
Processing Run Gradient Boosting
Processing Run CNN
423/423 [==============================] - 0s 190us/step
Processing Run Extra Trees

Got Sequence  1401

Number of users data retrieved: 6

Processing Run Bernoulli
Processing Run Random Forest
Processing Run G

Processing Run Extra Trees

Got Sequence  1431

Number of users data retrieved: 6

Processing Run Bernoulli
Processing Run Random Forest
Processing Run Gradient Boosting
Processing Run CNN
423/423 [==============================] - 0s 176us/step
Processing Run Extra Trees

Got Sequence  1432

Number of users data retrieved: 6

Processing Run Bernoulli
Processing Run Random Forest
Processing Run Gradient Boosting
Processing Run CNN
423/423 [==============================] - 0s 227us/step
Processing Run Extra Trees

Got Sequence  1433

Number of users data retrieved: 6

Processing Run Bernoulli
Processing Run Random Forest
Processing Run Gradient Boosting
Processing Run CNN
423/423 [==============================] - 0s 234us/step
Processing Run Extra Trees

Got Sequence  1434

Number of users data retrieved: 6

Processing Run Bernoulli
Processing Run Random Forest
Processing Run Gradient Boosting
Processing Run CNN
423/423 [==============================] - 0s 157us/step
Processing Run E


Processing Run Bernoulli
Processing Run Random Forest
Processing Run Gradient Boosting
Processing Run CNN
423/423 [==============================] - 0s 250us/step
Processing Run Extra Trees

Got Sequence  1465

Number of users data retrieved: 6

Processing Run Bernoulli
Processing Run Random Forest
Processing Run Gradient Boosting
Processing Run CNN
423/423 [==============================] - 0s 233us/step
Processing Run Extra Trees

Got Sequence  1466

Number of users data retrieved: 6

Processing Run Bernoulli
Processing Run Random Forest
Processing Run Gradient Boosting
Processing Run CNN
423/423 [==============================] - 0s 246us/step
Processing Run Extra Trees

Got Sequence  1467

Number of users data retrieved: 6

Processing Run Bernoulli
Processing Run Random Forest
Processing Run Gradient Boosting
Processing Run CNN
423/423 [==============================] - 0s 357us/step
Processing Run Extra Trees

Got Sequence  1468

Number of users data retrieved: 6

Processing Run 

Processing Run CNN
423/423 [==============================] - 0s 406us/step
Processing Run Extra Trees

Got Sequence  1498

Number of users data retrieved: 5

Processing Run Bernoulli
Processing Run Random Forest
Processing Run Gradient Boosting
Processing Run CNN
423/423 [==============================] - 0s 349us/step
Processing Run Extra Trees

Got Sequence  1499

Number of users data retrieved: 5

Processing Run Bernoulli
Processing Run Random Forest
Processing Run Gradient Boosting
Processing Run CNN
423/423 [==============================] - 0s 199us/step
Processing Run Extra Trees

Got Sequence  1500

Number of users data retrieved: 5

Processing Run Bernoulli
Processing Run Random Forest
Processing Run Gradient Boosting
Processing Run CNN
423/423 [==============================] - 0s 290us/step
Processing Run Extra Trees

Got Sequence  1501

Number of users data retrieved: 5

Processing Run Bernoulli
Processing Run Random Forest
Processing Run Gradient Boosting
Processing Run C

...

Processing Run CNN
423/423 [==============================] - 0s 193us/step
Processing Run Extra Trees

Got Sequence  1798

Number of users data retrieved: 3

Processing Run Bernoulli
Processing Run Random Forest
Processing Run Gradient Boosting
Processing Run CNN
423/423 [==============================] - 0s 224us/step
Processing Run Extra Trees

Got Sequence  1799

Number of users data retrieved: 3

Processing Run Bernoulli
Processing Run Random Forest
Processing Run Gradient Boosting
Processing Run CNN
423/423 [==============================] - 0s 135us/step
Processing Run Extra Trees

Got Sequence  1800

Number of users data retrieved: 3

Processing Run Bernoulli
Processing Run Random Forest
Processing Run Gradient Boosting
Processing Run CNN
423/423 [==============================] - 0s 298us/step
Processing Run Extra Trees

Got Sequence  1801

Number of users data retrieved: 3

Processing Run Bernoulli
Processing Run Random Forest
Processing Run Gradient Boosting
Processing Run C

Processing Run Extra Trees

Got Sequence  1831

Number of users data retrieved: 3

Processing Run Bernoulli
Processing Run Random Forest
Processing Run Gradient Boosting
Processing Run CNN
423/423 [==============================] - 0s 212us/step
Processing Run Extra Trees

Got Sequence  1832

Number of users data retrieved: 3

Processing Run Bernoulli
Processing Run Random Forest
Processing Run Gradient Boosting
Processing Run CNN
423/423 [==============================] - 0s 218us/step
Processing Run Extra Trees

Got Sequence  1833

Number of users data retrieved: 3

Processing Run Bernoulli
Processing Run Random Forest
Processing Run Gradient Boosting
Processing Run CNN
423/423 [==============================] - 0s 175us/step
Processing Run Extra Trees

Got Sequence  1834

Number of users data retrieved: 3

Processing Run Bernoulli
Processing Run Random Forest
Processing Run Gradient Boosting
Processing Run CNN
423/423 [==============================] - 0s 212us/step
Processing Run E


Processing Run Bernoulli
Processing Run Random Forest
Processing Run Gradient Boosting
Processing Run CNN
423/423 [==============================] - 0s 231us/step
Processing Run Extra Trees

Got Sequence  1865

Number of users data retrieved: 3

Processing Run Bernoulli
Processing Run Random Forest
Processing Run Gradient Boosting
Processing Run CNN
423/423 [==============================] - 0s 198us/step
Processing Run Extra Trees

Got Sequence  1866

Number of users data retrieved: 3

Processing Run Bernoulli
Processing Run Random Forest
Processing Run Gradient Boosting
Processing Run CNN
423/423 [==============================] - 0s 247us/step
Processing Run Extra Trees

Got Sequence  1867

Number of users data retrieved: 3

Processing Run Bernoulli
Processing Run Random Forest
Processing Run Gradient Boosting
Processing Run CNN
423/423 [==============================] - 0s 174us/step
Processing Run Extra Trees

Got Sequence  1868

Number of users data retrieved: 3

Processing Run 

Processing Run CNN
423/423 [==============================] - 0s 401us/step
Processing Run Extra Trees

Got Sequence  1898

Number of users data retrieved: 3

Processing Run Bernoulli
Processing Run Random Forest
Processing Run Gradient Boosting
Processing Run CNN
423/423 [==============================] - 0s 305us/step
Processing Run Extra Trees

Got Sequence  1899

Number of users data retrieved: 3

Processing Run Bernoulli
Processing Run Random Forest
Processing Run Gradient Boosting
Processing Run CNN
423/423 [==============================] - 0s 355us/step
Processing Run Extra Trees

Got Sequence  1900

Number of users data retrieved: 3

Processing Run Bernoulli
Processing Run Random Forest
Processing Run Gradient Boosting
Processing Run CNN
423/423 [==============================] - 0s 336us/step
Processing Run Extra Trees

Got Sequence  1901

Number of users data retrieved: 3

Processing Run Bernoulli
Processing Run Random Forest
Processing Run Gradient Boosting
Processing Run C

Processing Run Extra Trees

Got Sequence  1931

Number of users data retrieved: 2

Processing Run Bernoulli
Processing Run Random Forest
Processing Run Gradient Boosting
Processing Run CNN
423/423 [==============================] - 0s 235us/step
Processing Run Extra Trees

Got Sequence  1932

Number of users data retrieved: 2

Processing Run Bernoulli
Processing Run Random Forest
Processing Run Gradient Boosting
Processing Run CNN
423/423 [==============================] - 0s 222us/step
Processing Run Extra Trees

Got Sequence  1933

Number of users data retrieved: 2

Processing Run Bernoulli
Processing Run Random Forest
Processing Run Gradient Boosting
Processing Run CNN
423/423 [==============================] - 0s 207us/step
Processing Run Extra Trees

Got Sequence  1934

Number of users data retrieved: 2

Processing Run Bernoulli
Processing Run Random Forest
Processing Run Gradient Boosting
Processing Run CNN
423/423 [==============================] - 0s 207us/step
Processing Run E

In [98]:
maindf

,content,nick
0,dutchman say darn time beyond embarrassing bra...,subject4830
1,ll first time attending t find anything websit...,subject379
2,husband would let myoclonus trying train dogch...,subject1018
3,bipolar lovedont really criticism ill point mi...,subject9730
4,co sleeping ok broke month ago moved work bra...,subject4969
...,...,...
418,turn illidan pogchampimgurcomagfoimgurcomagfo...,subject5323
419,tall glass walldisney two type animal humanlik...,subject5056
420,make guy mad people telling trip planning irre...,subject1615
421,confirm quite game work control far loving ska...,subject299


In [99]:
maindf.to_csv('final-results.csv')

In [100]:
maindf['content'][0]

'dutchman say darn time beyond embarrassing branded taxpathetic embarrassing indeed surprisingwwwfrancecomenkaczynskicallslgbtrightsthreatpoland one would expect kind attitude middle east eugtonce country eu one really know apparently nowadays important keep country eu cost trying offend much actually defend eu supposed irony longer eu allows kinda behaviour attacking journalistswwwpoliticoeuarticlepolandspislawandjusticepartyaimstoregulatejournalistsmediaaftertheelection judgeseuobservercomjustice gayswwwindependentcouknewsworldeuropepolandlgbtfreezoneshomophobiahatespeechlawjusticepartyahtml continue bigger chance eu split twospeed union poland ee country want gtthere isnt supposed committee value respect human right letting anyone yes every country meet copenhagen criteriaeceuropaeuneighbourhoodenlargementpolicyglossarytermsaccessioncriteriaen want join union sad thing moment country actually join one give shit criterion anymore whatever want criterion bar enter union theyre dont ma

## The Program ran for a whole of 1 day 3 hrs and on 1990 rounds of data.